<a href="https://colab.research.google.com/github/shuen1106lu/Program-Language/blob/main/%E6%9C%9F%E6%9C%AB%E5%B0%88%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

專題介紹影片連結： https://youtu.be/bp-bTpEvU4U?si=_4TNTJKKtyLhJAUy

外部連結：使用的試算表

病人資料：https://docs.google.com/spreadsheets/d/1ucZMuaJx7Y7yBj8fm2A0MK0WTU4RyEKlDiDzqlmQKcA/edit?usp=sharing

藥品&照護：https://docs.google.com/spreadsheets/d/1fpCKNCE1HnCpdf4Exy7--6feYAI1w95YT3yJce8zo7w/edit?usp=sharing

輸出用：https://docs.google.com/spreadsheets/d/1VuxQ9dzJE0fpdgmHUdq0OMn5VkVxgsaHDtK6-njPbq4/edit?usp=sharing

In [ ]:
# 授權 Google Colab 存取 Google Sheets
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
import pandas as pd

病人資料：https://docs.google.com/spreadsheets/d/1ucZMuaJx7Y7yBj8fm2A0MK0WTU4RyEKlDiDzqlmQKcA/edit?usp=sharing

In [ ]:
gc = gspread.authorize(creds)
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets_1 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1ucZMuaJx7Y7yBj8fm2A0MK0WTU4RyEKlDiDzqlmQKcA/edit?usp=sharing')

gsheets_1

<Spreadsheet '期末專題-病人資料' id:1ucZMuaJx7Y7yBj8fm2A0MK0WTU4RyEKlDiDzqlmQKcA>

In [ ]:
from gspread_dataframe import set_with_dataframe

worksheet_1 = gsheets_1.get_worksheet(0)

worksheet_1

<Worksheet '工作表1' id:0>

In [ ]:
# 取得 Google Sheets 內容(病人)
data_1 = worksheet_1.get_all_values()  # 取得所有儲存格資料，回傳為 2D 陣列
df_1 = pd.DataFrame(data_1)

In [ ]:
df_1

,0,1,2,3,4,5,6
0,病人姓名,所在樓層,需要的藥物1,需要的藥物2,照護,照護等級,須注意的狀況
1,張三,1F,盤尼西林,無,無,1,咳嗽
2,李四,2F,嗎啡,抗過敏藥,協助翻身,1,無
3,王五,2F,化痰藥,抗過敏藥,協助拍背,2,咳嗽
4,趙六,3F,氣管舒張劑,無,無,2,氣喘
5,劉七,1F,葡萄糖點滴,盤尼西林,協助翻身,3,昏迷


藥品&照護：https://docs.google.com/spreadsheets/d/1fpCKNCE1HnCpdf4Exy7--6feYAI1w95YT3yJce8zo7w/edit?usp=sharing

In [ ]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets_2 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1fpCKNCE1HnCpdf4Exy7--6feYAI1w95YT3yJce8zo7w/edit?usp=sharing')

gsheets_2

<Spreadsheet '期末專題-藥品' id:1fpCKNCE1HnCpdf4Exy7--6feYAI1w95YT3yJce8zo7w>

In [ ]:
from gspread_dataframe import set_with_dataframe

worksheet_2 = gsheets_2.get_worksheet(0)

worksheet_2

<Worksheet '工作表1' id:0>

In [ ]:
# 取得 Google Sheets 內容(藥品)
data_2 = worksheet_2.get_all_values()  # 取得所有儲存格資料，回傳為 2D 陣列
df_2 = pd.DataFrame(data_2)

In [ ]:
df_2

,0,1,2
0,藥品名稱,藥品功效,使用間隔(次/每日)
1,盤尼西林,抑制病原體生長,2
2,嗎啡,止痛,1
3,化痰藥,通暢呼吸道,3
4,氣管舒張劑,強迫氣管擴張,3
5,葡萄糖點滴,給予養分,2
6,拍背(照護),協助氣管暢通,10
7,翻身(照護),避免長期壓迫形成褥瘡,8


In [ ]:
!pip install --upgrade gspread pandas

In [ ]:
from datetime import datetime, timedelta

In [ ]:
# 建立 06:00 ~ 23:00 的每小時時間點
time_slots = [datetime.strptime("06:00", "%H:%M") + timedelta(hours=i) for i in range(19)]  # 6:00 到 24:00

# 建立時段字典
schedule_by_hour = {t.strftime("%H:00"): [] for t in time_slots}

df_2 = pd.DataFrame(data_2[1:], columns=data_2[0])  # 使用第0列當欄位名稱

# 幫每個藥分配時間點
for index, row in df_2.iterrows():
    drug_name = row["藥品名稱"]
    times_per_day = int(row["使用間隔(次/每日)"])

    if times_per_day == 1:
        chosen_slots = [time_slots[len(time_slots) // 2]]
    else:
        interval = len(time_slots) / times_per_day
        chosen_slots = [
            time_slots[min(int(i * interval + interval / 2), len(time_slots) - 1)]
            for i in range(times_per_day)
        ]

    for slot in chosen_slots:
        key = slot.strftime("%H:00")
        schedule_by_hour[key].append(drug_name)

# 轉成 DataFrame
hourly_schedule = pd.DataFrame([
    {"時間": hour, "藥品清單/所需照護": ", ".join(drugs) if drugs else ""}
    for hour, drugs in schedule_by_hour.items()
])

hourly_schedule


,時間,藥品清單/所需照護
0,06:00,拍背(照護)
1,07:00,翻身(照護)
2,08:00,拍背(照護)
3,09:00,"化痰藥, 氣管舒張劑, 翻身(照護)"
4,10:00,"盤尼西林, 葡萄糖點滴, 拍背(照護)"
5,11:00,翻身(照護)
6,12:00,拍背(照護)
7,13:00,
8,14:00,"拍背(照護), 翻身(照護)"
9,15:00,"嗎啡, 化痰藥, 氣管舒張劑"


(4/24 路)藥品編排部分完成
試算表藥品部分新增拍背&翻身，名稱變更為藥品&照護

第一次大家的回饋：https://docs.google.com/document/d/1LdSaZv467K9Hgm_XS2im2ia-yOMIVdsBNKidJlF-yIw/edit?usp=sharing

第一次進度報告：https://youtu.be/guIjrDoQ0TM?si=NOLP8PgWMrc8YZa4